## 05 - Structured Output and Typing

**Key Concept**: LLMs generate text by default. Structured output forces them to return validated Python objects or JSON that downstream systems can reliably consume.

**What this covers:**
1. Why structured output matters for production systems
2. Defining schemas with Pydantic, dataclasses, TypedDict
3. Direct model-to-structured output (no agents)
4. Combining structured output with runnables
5. Structured output in agent workflows
6. Error handling and validation strategies


In [2]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# For structured output schemas
from pydantic import BaseModel, Field
from typing import Optional, List, Literal, Union
from dataclasses import dataclass
from typing_extensions import TypedDict

import json


/Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,  # Deterministic for consistent structured output
)


### Why Structured Output?

**The problem with free-form text:**
- Parsing is fragile: regex breaks on edge cases
- Downstream systems (DBs, APIs) need predictable formats
- Testing becomes harder without type guarantees

**Structured output provides:**
- Schema-enforced responses: model MUST return valid data
- Automatic validation: Pydantic catches type errors
- IDE support: autocomplete and type checking work

Think of it as a contract between your LLM and the rest of your application.


### Defining Output Schemas

Three common approaches, each with trade-offs:

**1. Pydantic models** (recommended)
- Rich validation, nested types, custom validators
- Best IDE support and documentation

**2. Dataclasses**
- Simpler syntax, built into Python
- Less validation power than Pydantic

**3. TypedDict**
- Lightweight, dict-like access
- Good for simple key-value structures


In [4]:
# Pydantic - the most powerful option
class MovieInfo(BaseModel):
    """Information extracted from a movie description."""
    title: str = Field(description="The movie title")
    year: int = Field(description="Release year")
    genre: Literal["action", "comedy", "drama", "sci-fi", "horror", "other"] = Field(
        description="Primary genre of the movie"
    )
    director: Optional[str] = Field(default=None, description="Director name if mentioned")
    rating: Optional[float] = Field(default=None, description="Rating out of 10 if mentioned", ge=0, le=10)

# Dataclass alternative
@dataclass
class TaskItem:
    """A task extracted from user input."""
    description: str
    priority: str  # "low", "medium", "high"
    due_date: Optional[str] = None

# TypedDict alternative
class ContactInfo(TypedDict):
    """Contact information extracted from text."""
    name: str
    email: str
    phone: Optional[str]

print("Schemas defined:")
print(f"  - MovieInfo (Pydantic): {list(MovieInfo.model_fields.keys())}")
print(f"  - TaskItem (dataclass): {[f for f in TaskItem.__dataclass_fields__.keys()]}")
print(f"  - ContactInfo (TypedDict): {list(ContactInfo.__annotations__.keys())}")


Schemas defined:
  - MovieInfo (Pydantic): ['title', 'year', 'genre', 'director', 'rating']
  - TaskItem (dataclass): ['description', 'priority', 'due_date']
  - ContactInfo (TypedDict): ['name', 'email', 'phone']


### Direct Model to Structured Output

The simplest pattern: call `model.with_structured_output(Schema)` and get back a Python object.

**How it works:**
1. LangChain converts your schema to JSON schema
2. Model is instructed to return data matching that schema
3. Response is parsed and validated automatically

No agents, no tools - just direct extraction.


In [5]:
# Bind schema to model
structured_llm = llm.with_structured_output(MovieInfo)

# Extract structured data from text
movie_text = """
I just watched Inception (2010) directed by Christopher Nolan. 
It's a mind-bending sci-fi thriller about dreams within dreams.
I'd give it a solid 9 out of 10.
"""

result = structured_llm.invoke(movie_text)

print(f"Type: {type(result).__name__}")
print(f"\nExtracted MovieInfo:")
print(f"  Title: {result.title}")
print(f"  Year: {result.year}")
print(f"  Genre: {result.genre}")
print(f"  Director: {result.director}")
print(f"  Rating: {result.rating}")


Type: MovieInfo

Extracted MovieInfo:
  Title: Inception
  Year: 2010
  Genre: sci-fi
  Director: Christopher Nolan
  Rating: 9.0


In [6]:
# Handling missing information gracefully
# Optional fields stay None when data isn't present

partial_text = "The Dark Knight is an amazing action movie from 2008."

result = structured_llm.invoke(partial_text)

print("Partial extraction (missing director, rating):")
print(f"  Title: {result.title}")
print(f"  Year: {result.year}")
print(f"  Genre: {result.genre}")
print(f"  Director: {result.director}")
print(f"  Rating: {result.rating}")


Partial extraction (missing director, rating):
  Title: The Dark Knight
  Year: 2008
  Genre: action
  Director: None
  Rating: None


### Validation in Action

Pydantic validates data at parse time. If the model returns invalid data:
- Type mismatches raise `ValidationError`
- Constraint violations (ge, le, regex) are caught
- Required fields must be present

This catches errors early, before they propagate through your system.


In [7]:
# Schema with strict constraints
class ProductReview(BaseModel):
    """A product review with validated fields."""
    product_name: str = Field(description="Name of the product")
    rating: int = Field(description="Rating from 1 to 5", ge=1, le=5)
    sentiment: Literal["positive", "negative", "neutral"] = Field(
        description="Overall sentiment of the review"
    )
    key_points: List[str] = Field(
        description="Main points from the review, max 3 items",
        max_length=3
    )

review_llm = llm.with_structured_output(ProductReview)

review_text = """
Just got the Sony WH-1000XM5 headphones. The noise cancellation is incredible,
battery life is great, but they're a bit pricey. Overall very happy with the purchase.
Would rate them 4 out of 5.
"""

review = review_llm.invoke(review_text)

print(f"Product: {review.product_name}")
print(f"Rating: {review.rating}/5")
print(f"Sentiment: {review.sentiment}")
print(f"Key points:")
for point in review.key_points:
    print(f"  - {point}")


Product: Sony WH-1000XM5 headphones
Rating: 4/5
Sentiment: positive
Key points:
  - Incredible noise cancellation
  - Great battery life
  - A bit pricey


### Combining Structured Output with Runnables

Structured output fits naturally into LangChain's Runnable pipeline pattern.

**Pipeline flow:**
```
Input text -> Prompt template -> Model (structured) -> Python object
```

The output object can then feed directly into:
- Database writes
- API calls
- Further model calls


In [10]:
# Schema for meeting notes extraction
class MeetingNotes(BaseModel):
    """Structured meeting notes."""
    title: str = Field(description="Meeting title or topic")
    attendees: List[str] = Field(description="List of attendee names")
    action_items: List[str] = Field(description="List of action items")
    next_meeting: Optional[str] = Field(default=None, description="Next meeting date if mentioned")

# Build a reusable extraction pipeline
extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract structured meeting information from the provided notes. Be precise and only include information explicitly mentioned."),
    ("human", "{text}")
])

# Chain: prompt -> structured model
meeting_extractor = extraction_prompt | llm.with_structured_output(MeetingNotes)

# Test the pipeline
raw_notes = """
Q3 Planning Meeting - July 15th
Present: Sarah, Mike, Jennifer, David

Discussed roadmap priorities for Q3. Sarah will finalize the budget by Friday.
Mike to coordinate with engineering on the new feature timeline.
Jennifer will prepare the customer presentation for next week.

Next sync scheduled for July 22nd.
"""

notes = meeting_extractor.invoke({"text": raw_notes})

print(f"Meeting: {notes.title}")
print(f"Attendees: {', '.join(notes.attendees)}")
print(f"\nAction Items:")
for item in notes.action_items:
    print(f"  - {item}")
print(f"\nNext Meeting: {notes.next_meeting}")


Meeting: Q3 Planning Meeting
Attendees: Sarah, Mike, Jennifer, David

Action Items:
  - Sarah will finalize the budget by Friday
  - Mike to coordinate with engineering on the new feature timeline
  - Jennifer will prepare the customer presentation for next week

Next Meeting: July 22nd


In [11]:
# Batch processing with structured output
# The same pipeline works with .batch() for multiple inputs

meeting_texts = [
    {"text": "Sprint Retro - Team Alpha. Attendees: John, Lisa. Action: John to update docs."},
    {"text": "1:1 with Manager. Present: You, Boss. Follow-up: Submit self-review by Monday."},
]

results = meeting_extractor.batch(meeting_texts)

print("Batch extraction results:")
for i, notes in enumerate(results, 1):
    print(f"\n{i}. {notes.title}")
    print(f"   Attendees: {notes.attendees}")
    print(f"   Actions: {notes.action_items}")


Batch extraction results:

1. Sprint Retro - Team Alpha
   Attendees: ['John', 'Lisa']
   Actions: ['John to update docs']

2. 1:1 with Manager
   Attendees: ['You', 'Boss']
   Actions: ['Submit self-review by Monday']


### Structured Output in Agent Workflows

Agents can return structured output as their final response. Two approaches:

**1. Native structured output (response_format)**
- Agent directly returns the schema
- Requires model support for structured output

**2. Two-step: Agent text -> Formatter model**
- Agent returns free-form text
- Second model converts to structured format
- Works with any model


### Response Format
Controls how the agent returns structured data:
- `ToolStrategy[StructuredResponseT]`: Uses tool calling for structured output
- `ProviderStrategy[StructuredResponseT]`: Uses provider-native structured output
- `type[StructuredResponseT]`: Schema type - automatically selects best strategy based on model capabilities
- `None`: No structured output

In [12]:
from langchain.agents import create_agent
from langchain.tools import tool

# Define output schema for agent
class ResearchResult(BaseModel):
    """Structured research findings."""
    topic: str = Field(description="The research topic")
    findings: List[str] = Field(description="Key findings from research")
    data_sources: List[str] = Field(description="Sources of data used")
    confidence: Literal["low", "medium", "high"] = Field(
        description="Confidence level in the findings"
    )

# Simple research tools
@tool
def search_database(query: str) -> str:
    """Search internal database for information."""
    # Simulated database
    data = {
        "sales": "Q3 sales: $2.4M, up 15% YoY. Top product: Widget Pro.",
        "customers": "Active customers: 1,247. Churn rate: 3.2%. NPS: 72.",
        "inventory": "Stock levels healthy. Widget Pro: 500 units. Widget Basic: 1200 units."
    }
    for key, value in data.items():
        if key in query.lower():
            return value
    return "No matching data found."

@tool
def get_market_report(sector: str) -> str:
    """Get market report for a sector."""
    return f"Market report for {sector}: Growth rate 8%, competition increasing, favorable regulations."

# Create agent with structured output
research_agent = create_agent(
    llm,
    tools=[search_database, get_market_report],
    system_prompt="""You are a research assistant. Use the available tools to gather information,
then return your findings in the structured format specified.""",
    response_format=ResearchResult
)

print("Research agent created with structured output")


Research agent created with structured output


In [13]:
# Run the agent - it will use tools and return structured data
result = research_agent.invoke({
    "messages": [{"role": "user", "content": "Research our current sales performance and customer metrics."}]
})
print(result)
# Access structured response
if "structured_response" in result:
    research = result["structured_response"]
    print(f"Topic: {research.topic}")
    print(f"\nFindings:")
    for finding in research.findings:
        print(f"  - {finding}")
    print(f"\nData Sources: {research.data_sources}")
    print(f"Confidence: {research.confidence}")
else:
    # Fallback: check last message content
    print("Agent response:")
    print(result["messages"][-1].content)


{'messages': [HumanMessage(content='Research our current sales performance and customer metrics.', additional_kwargs={}, response_metadata={}, id='5caa90b6-b182-450e-8900-a6678a7d12ed'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 's5qx2p214', 'function': {'arguments': '{"query":"current sales performance and customer metrics"}', 'name': 'search_database'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 894, 'total_tokens': 928, 'completion_time': 0.051710421, 'completion_tokens_details': None, 'prompt_time': 0.018556597, 'prompt_tokens_details': None, 'queue_time': 0.052474922, 'total_time': 0.070267018}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--fcc5683a-6943-4c11-9440-b938792ec7d5-0', tool_calls=[{'name': 'search_database', 'args': {'quer

### `ProviderStrategy[StructuredResponseT]`
- ProviderStrategy uses the LLM provider’s native support for structured output. That means the provider/model API itself has some mechanism (e.g. JSON mode, function-calling, “structured response” mode) to generate a well-formed JSON (or other structured) output that matches a schema.
- This is ideal if your chosen model/provider supports structured output natively

### `ToolStrategy[StructuredResponseT]` 
- ToolStrategy uses tool-calling under the hood to enforce structured output. That means LangChain treats the structured response as if it were the “arguments” of a (virtual) tool call, even if no real external side-effect happens. The LLM is prompted to produce a “tool call” with arguments matching your schema, and LangChain captures the arguments as structured data.
- Useful when the model/provider does not support native structured output. In that case, ToolStrategy gives you a fallback mechanism so that you still can get structured data, even though the model may only output free-form text — because you treat the output as a “tool call.”

Note: If the provider natively supports structured output for your model choice, it is functionally equivalent to write response_format=ProductReview instead of response_format=ProviderStrategy(ProductReview). In either case, if structured output is not supported, the agent will fall back to a tool calling strategy.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ProductReview(BaseModel):
    """Analysis of a product review."""
    rating: int | None = Field(description="The rating of the product", ge=1, le=5)
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
    key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")

agent = create_agent(
    llm,
    response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"]

ProductReview(rating=5, sentiment='positive', key_points=['fast shipping', 'expensive'])

In [21]:
agent = create_agent(
    llm,
    response_format=ToolStrategy(schema=ProductReview, tool_message_content="Action item captured and sentiment updated!"),
)   

result = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
for msg in result["messages"]:
   msg.pretty_print()


================================ Human Message =================================

Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'
================================== Ai Message ==================================
Tool Calls:
  ProductReview (dr37ceyvg)
 Call ID: dr37ceyvg
  Args:
    key_points: ['fast shipping', 'expensive']
    rating: 5
    sentiment: positive
================================= Tool Message =================================
Name: ProductReview

Action item captured and sentiment updated!



### Multiple schema outputs

In [25]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    email: str = Field(description="Email address")

class EventDetails(BaseModel):
    """Details of an event."""
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Event date")

agent = create_agent(
    llm,
    response_format=ToolStrategy(Union[ContactInfo, EventDetails])  # Default: handle_errors=True
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th"}]
})
result["structured_response"]

ContactInfo(name='John Doe', email='john@email.com')

### Error handling
Models can make mistakes when generating structured output via tool calling. LangChain provides intelligent retry mechanisms to handle these errors automatically.

- When structured output doesn’t match the expected schema, the agent provides specific error feedback
- You can customize how errors are handled using the `handle_errors` parameter.
- If `handle_errors` is a string, the agent will always prompt the model to re-try with a fixed tool message.
- We can have a Custom error handler function as well in `handle_errors`